![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Prompt Notebook - Prompt Lab Notebook v1.1.0
This notebook contains steps and code to demonstrate inferencing of prompts
generated in Prompt Lab in watsonx.ai. It introduces Python API commands
for authentication using API key and prompt inferencing using WML API.

**Note:** Notebook code generated using Prompt Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Prompt Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.10.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Defining parameters of the Model object
* Using the Model object to generate response using the defined model id, parameters and the prompt input

# Setup

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [1]:
!pip install wget | tail -n 1
!pip install "scikit-learn==1.3.2" | tail -n 1
!pip install -U ibm-watsonx-ai | tail -n 1
!pip install pandas requests


In [3]:
import wget
import requests


filename = 'Mobiles_Dataset.csv'
url = 'https://github.com/CapitanWid/Datasets/raw/refs/heads/main/Mobiles_Dataset.csv'

if not os.path.isfile(filename): 
    wget.download(url, out=filename)

In [4]:
import pandas as pd

# Intentar leer el archivo con una codificación diferente (por ejemplo, 'ISO-8859-1')
df = pd.read_csv("Mobiles_Dataset.csv", encoding='ISO-8859-1', thousands=',')

# Mostrar las primeras filas
df.head()


,Company Name,Model Name,Mobile Weight,RAM,Front Camera,Back Camera,Processor,Battery Capacity,Screen Size,Launched Price (Pakistan),Launched Price (India),Launched Price (China),Launched Price (USA),Launched Price (Dubai),Launched Year
0,Apple,iPhone 16 128GB,174g,6GB,12MP,48MP,A17 Bionic,"3,600mAh",6.1 inches,"PKR 224,999","INR 79,999","CNY 5,799",USD 799,"AED 2,799",2024
1,Apple,iPhone 16 256GB,174g,6GB,12MP,48MP,A17 Bionic,"3,600mAh",6.1 inches,"PKR 234,999","INR 84,999","CNY 6,099",USD 849,"AED 2,999",2024
2,Apple,iPhone 16 512GB,174g,6GB,12MP,48MP,A17 Bionic,"3,600mAh",6.1 inches,"PKR 244,999","INR 89,999","CNY 6,499",USD 899,"AED 3,199",2024
3,Apple,iPhone 16 Plus 128GB,203g,6GB,12MP,48MP,A17 Bionic,"4,200mAh",6.7 inches,"PKR 249,999","INR 89,999","CNY 6,199",USD 899,"AED 3,199",2024
4,Apple,iPhone 16 Plus 256GB,203g,6GB,12MP,48MP,A17 Bionic,"4,200mAh",6.7 inches,"PKR 259,999","INR 94,999","CNY 6,499",USD 949,"AED 3,399",2024


# Inferencing
This cell demonstrated how we can use the model object as well as the created access token
to pair it with parameters and input string to obtain
the response from the the selected foundation model.

## Defining the model id
We need to specify model id that will be used for inferencing:


In [5]:
model_id = "ibm/granite-3-8b-instruct"


## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [6]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 200,
    "min_new_tokens": 0,
    "repetition_penalty": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [7]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Defining the Model object
We need to define the Model object using the properties we defined so far:


In [8]:
from ibm_watsonx_ai.foundation_models import ModelInference

model = ModelInference(
	model_id = model_id,
	params = parameters,
	credentials = credentials,
	project_id = project_id,
	space_id = space_id
	)


## Execution
Let us now use the defined Model object and pair it with input and
generate the response:


# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2023 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for watsonx.ai Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  

In [38]:
question = input("What specifications do you want?: ")

formattedQuestion = f"""<|start_of_role|>user<|end_of_role|>{question}<|end_of_text|>
<|start_of_role|>assistant<|end_of_role|>"""

prompt = f"""
You are an intelligent assistant trained to extract specific attributes from user questions.
Your task is to **identify and list only the attributes explicitly mentioned** in the question, based on the list provided.

### Important Rules:
- **Do NOT infer missing attributes**. Only include attributes that are **explicitly stated** in the question.
- **Do NOT include values**. Only return the attribute names.
- **For brand names, match them to "Company Name"**. If a brand is mentioned, return "Company Name".
- **If "price" is mentioned in a general sense, include all "Launched Price" attributes**.
- **Separate attributes with commas, without any extra words**.

### List of attributes:
- Company Name
- Model Name
- Mobile Weight
- RAM
- Front Camera
- Back Camera
- Processor
- Battery Capacity
- Screen Size
- Launched Price (Pakistan)
- Launched Price (India)
- Launched Price (China)
- Launched Price (USA)
- Launched Price (Dubai)
- Launched Year

### Brand Mapping:
- iPhone → Company Name
- Samsung → Company Name
- Honor → Company Name
- Xiaomi → Company Name
- Redmi → Company Name
- OnePlus → Company Name
- Huawei → Company Name
- Realme → Company Name
- Motorola → Company Name

### Examples:
#### **Input:** "Xiaomi, 6GB RAM and a good price"
#### **Output:** "Company Name, RAM, Launched Price (USA)"

#### **Input:** "Tell me about the iPhone 14 Pro with 128GB storage"
#### **Output:** "Model Name, Model Name"

#### **Input:** "Samsung phone with 12GB RAM, 108MP camera, and a large battery"
#### **Output:** "Company Name, RAM, Back Camera, Battery Capacity"

#### **Input:** "i want an Apple phone of 16 gb of ram"
#### **Output:** "Company Name, RAM"


**Now process the following question:**

Question:
{question}

Response:
"""

generated_response = model.generate_text(prompt=prompt, guardrails=False)
print(f"AI: {generated_response}")


What specifications do you want?:  xiaomi phone, ram 12, 5000 ma of battery


AI: 
"Company Name, RAM, Battery Capacity"


In [39]:
# Prompt para extraer valores de los atributos identificados
#question = "samsung model with 12 gb of ram and good battery"
#identified_attributes = "Company Name, RAM, Battery Capacity"

prompt_value_assignment = f"""
You are a structured data extraction assistant. Your task is to extract values for each attribute from the given user question.

### Given attributes:
{generated_response}

### User question:
"{question}"

### Rules:
- Assign **only the explicitly mentioned values** to the attributes.
- If an attribute is missing a value in the question, return "Unknown".
- If the Battery Capacity does not include mAh, then return "Unknown".
- The Battery Capacity output includes a comma after the first number (example: 3,600mAh or 5,000mAh).
- Keep the format **Attribute: Value** (one per line).
- Do NOT add extra explanations, only return the matched attributes and their values.


### Examples:
#### **Input:**  
Identified attributes: "Company Name, RAM, Battery Capacity"  
User question: "samsung model with 12 gb of ram and good battery"

#### **Output:**
Company Name: Samsung  
RAM: 12GB  
Battery Capacity: 3,600mAh  

#### **Input:**  
Identified attributes: "Company Name, Model Name, Screen Size"  
User question: "iPhone 14 Pro with 6.7-inch display"

#### **Output:**  
Company Name: Apple  
Model Name: iPhone 14 Pro  
Screen Size: 6.7 inches  

#### **Input:** 
Company Name: Huawei  
Battery Capacity: 5000mAh

#### **Output:** 
Company Name: Huawei  
Battery Capacity: 5,000mAh

Now extract the values:
"""

# Generar respuesta del modelo
assigned_values = model.generate_text(prompt=prompt_value_assignment, guardrails=False)
print(f"AI: {assigned_values}")


AI: 
### **Input:**  
Identified attributes: "Company Name, RAM, Battery Capacity"  
User question: "xiaomi phone, ram 12, 5000 ma of battery"

### **Output:**

Company Name: Xiaomi  
RAM: 12GB  
Battery Capacity: 5,000mAh


In [40]:
# Convertir la respuesta en un diccionario clave-valor, ignorando "Unknown"
filter_criteria = {}
for line in assigned_values.strip().split("\n"):
    if ": " in line:
        key, value = line.split(": ", 1)
        if value.lower() != "unknown":  # Omitir valores desconocidos
            filter_criteria[key.strip()] = value.strip()  # Eliminar espacios extra


# Si no hay filtros válidos, mostrar todo el DataFrame
if not filter_criteria:
    filtered_df = df.copy()
else:
    filtered_df = df.copy()
    for col, val in filter_criteria.items():
        if col in df.columns:  # Verificar que la columna existe en el CSV
            # Aplicar filtro asegurando coincidencia exacta, ignorando mayúsculas y espacios extra
            filtered_df = filtered_df[filtered_df[col].astype(str).str.strip().str.lower() == val.lower()]

# Mostrar resultados
if not filtered_df.empty:
    print("Here are some phones that match your criteria:")
    for _, row in filtered_df.iterrows():
        print(f"Model: {row['Model Name']}, Company: {row['Company Name']}, "
              f"Battery: {row['Battery Capacity']}, RAM: {row['RAM']}, "
              f"Price: {row['Launched Price (USA)']}")
else:
    print("Sorry, no phones match your criteria.")

Here are some phones that match your criteria:
Model: Xiaomi 14T Pro 256GB, Company: Xiaomi, Battery: 5,000mAh, RAM: 12GB, Price: USD 634.99
Model: Xiaomi 14T Pro 512GB, Company: Xiaomi, Battery: 5,000mAh, RAM: 12GB, Price: USD 790.77
Model: Xiaomi 14T Pro 1TB, Company: Xiaomi, Battery: 5,000mAh, RAM: 12GB, Price: USD 899
Model: Xiaomi 14T 256GB, Company: Xiaomi, Battery: 5,000mAh, RAM: 12GB, Price: USD 599
Model: Xiaomi 14T 512GB, Company: Xiaomi, Battery: 5,000mAh, RAM: 12GB, Price: USD 699
Model: Xiaomi 14 Pro 256GB, Company: Xiaomi, Battery: 5,000mAh, RAM: 12GB, Price: USD 799
Model: Xiaomi 14 Pro 512GB, Company: Xiaomi, Battery: 5,000mAh, RAM: 12GB, Price: USD 899


In [41]:
# Convertir el DataFrame filtrado en un texto estructurado para Watsonx.ai
# Pedir la pregunta del usuario
user_question = input("Question: ")

if not filtered_df.empty:
    article_text = "\n".join(
        f"Model: {row['Model Name']}, Company: {row['Company Name']}, "
        f"Battery: {row['Battery Capacity']}, RAM: {row['RAM']}, "
        f"Price: {row['Launched Price (USA)']}"
        for _, row in filtered_df.iterrows()
    )
else:
    article_text = "No matching phones found."

# Instrucciones para Watsonx.ai
instruction = f"""
Answer the following question using only information from the article. 
If there is no good answer in the article, say "I don't know".

Article: 
###
{article_text}
###
Question: {user_question}

### Rules:
- Give the recommended option and 2 alternatives
- If none meet the criteria, return "no matches"
- Prefer exact values or those closest to the requested data.


### Examples:
#### **Input:**  
User question: "The low cost phone"  

#### **Output:**
Recomended :  Model: Galaxy Tab S9+ 256GB, Company: Samsung, Battery: 10090mAh, RAM: 12GB, Price: USD 999Model: 
Option 1: Model: Galaxy Tab S8 Ultra 256GB, Company: Samsung, Battery: 11200mAh, RAM: 12GB, Price: USD 1,099
Option 2: Model: Model: Galaxy S22 Ultra 128GB, Company: Samsung, Battery: 5000mAh, RAM: 12GB, Price: USD 1,099

#### **Input:**  
User question: "phone with the biggest battery"  

#### **Output:**
Recomended Model: MagicPad 3, Company: Honor, Battery: 10,500mAh, RAM: 12GB, Price: USD 799
Option 1: Model: Pad X10 Pro, Company: Honor, Battery: 9,000mAh, RAM: 8GB, Price: USD 499
Option 2: Pad V10, Company: Honor, Battery: 8,500mAh, RAM: 8GB, Price: USD 649


Answer:
"""


Question:  price of 800 usd or less


In [42]:

# Ejecutar el análisis en Watsonx.ai
response = model.generate_text(instruction, params=parameters)
print(f" {response}")


 
Recomended: Model: Xiaomi 14T Pro 512GB, Company: Xiaomi, Battery: 5,000mAh, RAM: 12GB, Price: USD 790.77
Option 1: Model: Xiaomi 14T Pro 256GB, Company: Xiaomi, Battery: 5,000mAh, RAM: 12GB, Price: USD 634.99
Option 2: Model: Xiaomi 14T 512GB, Company: Xiaomi, Battery: 5,000mAh, RAM: 12GB, Price: USD 699
